In [ ]:
import init
from six import print_
from proceso import OUTPUT_DIR, ROOT_DIR, SHP_DIR, PARAGUAY_SHP_URL
import os
from os import path as pth
import logging
import sys
import pandas as pd
from climatepy.scenario.graph import draw_bar_xls

log = logging.getLogger()
log.handlers.append(logging.StreamHandler(stream=sys.__stdout__))
log.setLevel(logging.INFO)

DATA_DIR = pth.join(OUTPUT_DIR, "PARAGUAY")
RCP_FILE = [
    pth.join(DATA_DIR, "RCP4.5", "mensual_prec_py.xlsx"),
    pth.join(DATA_DIR, "RCP4.5", "mensual_temp_py.xlsx"),
    pth.join(DATA_DIR, "RCP8.5", "mensual_prec_py.xlsx"),
    pth.join(DATA_DIR, "RCP8.5", "mensual_temp_py.xlsx")
    ]
CRU_FILE = pth.join(DATA_DIR, "mensual_cru_py.xlsx")
ETA_FILE = pth.join(DATA_DIR, "mensual_eta_py.xlsx")
config = pth.join(ROOT_DIR, "proceso", "line_config.ini")

OUTPUT_BAR = pth.join(DATA_DIR, "compara")
if not pth.exists(OUTPUT_BAR):
    os.makedirs(OUTPUT_BAR)

In [ ]:
eta_pre_ds = pd.read_excel(RCP_FILE[0], index_col=0, header=0, parse_dates=True, sheetname="1961_1990")
eta_tmp_ds = pd.read_excel(RCP_FILE[1], index_col=0, header=0, parse_dates=True, sheetname="1961_1990")
cru_pre_ds = pd.read_excel(CRU_FILE, index_col=0, header=0, parse_dates=True, sheetname="PRE")
cru_tmp_ds = pd.read_excel(CRU_FILE, index_col=0, header=0, parse_dates=True, sheetname="TMP")

deps = ~eta_pre_ds.columns.isin((u"Asunción",))
meses = cru_pre_ds.columns
eta_pre_ds = eta_pre_ds.iloc[:,deps]
eta_tmp_ds = eta_tmp_ds.iloc[:,deps]

eta_pre_ave = pd.DataFrame(index=cru_pre_ds.index, columns=meses)
eta_tmp_ave = pd.DataFrame(index=cru_pre_ds.index, columns=meses)

In [ ]:
# Calcular meses
def get_by_month(m, index): 
    return index.isin([dt for dt in index if dt.month == m])

for idx, mes in enumerate(meses):
    eta_pre_ave.values[:,idx] = eta_pre_ds.iloc[get_by_month(idx+1, eta_pre_ds.index),:].mean().T
    eta_tmp_ave.values[:,idx] = eta_tmp_ds.iloc[get_by_month(idx+1, eta_tmp_ds.index),:].mean().T

with pd.ExcelWriter(ETA_FILE) as writer:
    eta_pre_ave.to_excel(writer, sheet_name="PREC")
    eta_tmp_ave.to_excel(writer, sheet_name="TP2M")

In [ ]:
log.setLevel(logging.INFO)
/draw_bar_xls config \
               CRU_FILE ETA_FILE "pre" "prec" \
               pth.join(SHP_DIR PARAGUAY_SHP_URL) \
               OUTPUT_BAR "%(HASC_1)s" "%(NAME_1)s" 1 "Paraguay"

/draw_bar_xls config \
               CRU_FILE ETA_FILE "tmp" "tp2m" \
               pth.join(SHP_DIR PARAGUAY_SHP_URL) \
               OUTPUT_BAR  "%(HASC_1)s" "%(NAME_1)s" 1 "Paraguay"